# Advanced concepts of transformer architecture

### Decoder models

* transforsmers were developed for language translation, they usually consist of two parts
    * encoder -> process input data (source language sentence)
    * decoder -> generates output data (translated sentence)
* over time, decodes have become central to text generation tasks, forming basis of models like GPT, LLaMA

* decoder model
    * generative pre-training - > model is trained to predict the next word in a sequence based on previous words
    * autoregressive models -> sequentially generating text, predicting text on words that came before

* fine-tuning & reinforcement learning
    * after initial training, GPT models can be fine-tuned for specific tasks (question answering or classification), model is trained on labeled data (supervised learning)
    * reinforcement learning from human feedback is a fine-tuning method where human feedback is used to improve the performance, especially in apps like chatbots

* decoders in text generation
    * key difference from translation is that decoders rely on input from encoders (using cross-attention), in text generation they are independent, predicting next word based on preciding sequence
    * autoregressive process starts with begining-of-sentence token, it predicts the next word, and appends it to the sequence.

* masked self-attention in decoders hiddes future tokens

* text generation process
    * input prompt
    * tokenization
    * word embeddings
    * positional encoding
    * contextual embeddings
    * logits
    * argmax
    * appending & repeat
    * generation stops once end-of-sequence token or token limit is hit

### Encoder models

### Applications for language translation